In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import pyopencl as cl
import pyopencl.array
import pyopencl.algorithm
import matplotlib.pyplot as plt
norm = lambda x: np.max(np.abs(x))

In [ ]:
ctx = cl.create_some_context()
queue = cl.CommandQueue(ctx)

In [ ]:
with open("tensor.cl") as f:
    code = f.read()

prg = cl.Program(ctx, code).build()

matvec_single = prg.matvec_single
matvec_single.set_scalar_arg_dtypes([None, None, None, np.int32])

matmat_single = prg.matmat_single
matmat_single.set_scalar_arg_dtypes([None, None, None, 
                                     np.int32, np.int32])


In [ ]:
n = 16*10**2
k = 10

x_d = cl.array.arange(queue, n, dtype=np.double)/n
A_d = cl.array.arange(queue, n*n, dtype=np.double).reshape((n,n))/(n*n)
M_d = cl.array.arange(queue, n*k, dtype=np.double).reshape((n,k))/(n*k)
x = x_d.get()
A = A_d.get()
M = M_d.get()
b = A.dot(x)
B = A.dot(M)

## Matvec Single

In [ ]:
b_d = cl.array.Array(queue, n, dtype=np.double)
evt = matvec_single(queue, (4,), None, 
                    A_d.data, x_d.data, b_d.data, n)

norm(b_d.get()-b)

## Matmat Single

In [ ]:
B_d = cl.array.Array(queue, n*k, dtype=np.double)
evt = matmat_single(queue, (4,), None, 
                    A_d.data, M_d.data, B_d.data, n, k)

norm(B_d.get()-B.ravel())